In [1]:
import os

In [2]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    train_data_path: Path
    test_data_path: Path
    mismatch_data_type_path: Path

In [6]:
from salesPrediction.constants import *
from salesPrediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])       

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            mismatch_data_type_path = config.mismatch_data_type_path
        )

        return data_transformation_config
 

In [8]:
import os
from salesPrediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from salesPrediction.config.configuration import DataTransformationConfig
import json

In [ ]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        # we already have train and test data set in the resources folder, we only need to copy them 
        self.train_data = pd.read_csv(self.config.train_data_path)
        self.test_data = pd.read_csv(self.config.test_data_path)
    
    def train_test_splitting(self):

        print(self.train_data.dtypes)
        self.train_data.to_csv(os.path.join(self.config.data_path, "Train.csv"), index=False)
        self.test_data.to_csv(os.path.join(self.config.data_path, "Test.csv"), index= False)

        logger.info("Completed saving train and test data to transformation folder")
        logger.info(self.train_data.shape)
        logger.info(self.test_data.shape)

        print(self.train_data.shape)
        print(self.test_data.shape)
    
    def convert_column_datatype(self):
        try:

            with open(self.config.mismatch_data_type_path, "r") as f:
                mismatched_cols =  json.loads(f.read())

            data = pd.read_csv(self.config.train_data_path)
            for col, dtype_info in mismatched_cols.items():
                if dtype_info["expected"] == "string":
                    self.train_data[col] = self.train_data[col].astype("string")
                    logger.info(f"Converted data type of column {col} to {dtype_info['expected']}")

        except Exception as e:
            raise e

In [10]:
from salesPrediction.utils.common import get_validation_status

In [ ]:
try:
    # check if data type mismatch present
    validation_status = get_validation_status()
    logger.info(f" validation status ::  {validation_status}")

    if validation_status != False:

        config = ConfigurationManager()
        data_transformation_config = config.get_data_transformation_config()
        data_transformation = DataTransformation(config=data_transformation_config)

        # check if any datatype mismatch has occurred
        if validation_status == "DATATYPE_MISMATCH":            
            data_transformation.convert_column_datatype()            
        
        data_transformation.train_test_splitting()

except Exception as e:
    raise e


DATATYPE_MISMATCH
[2025-03-03 12:16:03,727: 174169248: validation status ::  DATATYPE_MISMATCH]
[2025-03-03 12:16:03,727: common:yaml file: config\config.yaml loaded successfully]
[2025-03-03 12:16:03,727: common:yaml file: params.yaml loaded successfully]
[2025-03-03 12:16:03,743: common:yaml file: schema.yaml loaded successfully]
[2025-03-03 12:16:03,745: common:created directory at artifacts]
[2025-03-03 12:16:03,746: common:created directory at artifacts/data_transformation]
hjahjdahsjdh DATATYPE_MISMATCH
{'expected': 'string', 'actual': 'object'}
[2025-03-03 12:16:03,780: 2561470484:Converted data type of column Item_Identifier to string]
{'expected': 'string', 'actual': 'object'}
[2025-03-03 12:16:03,792: 2561470484:Converted data type of column Item_Fat_Content to string]
{'expected': 'string', 'actual': 'object'}
[2025-03-03 12:16:03,794: 2561470484:Converted data type of column Item_Type to string]
{'expected': 'string', 'actual': 'object'}
[2025-03-03 12:16:03,796: 2561470484